In [ ]:

import os
from os.path import join
from praatio import textgrid
from praatio import praat_scripts
from praatio.utilities.constants import (
    Interval)

praat_exe = "/Applications/Praat.app/Contents/MacOS/Praat"
root = "/Users/sarah/Git/eesti_regilaul_corpus/audio"
input_wavs = join(root,"clicks")
input_tg = join(root, "ictus_tier")
output_tg = join(root,"lyric_tier")

SILENCE_LABEL = "x"
SOUND_LABEL = "ictus"

for fn in os.listdir(root):
        if ".wav" not in fn:
            continue
        fn.beats_to_textgird()
        #then, add the phrase tier
        if ".TextGrid" not in fn:
            continue
        tg = textgrid.openTextgrid(fn,True)
        ictus_dict = tg.tierDict
        myTier = textgrid.IntervalTier("phrase",entryList= ictus_dict[1])
        lyric_lines = open(join(root,fn,".txt")).readlines()
        for interval, line in myTier, lyric_lines:
            #TODO first edge case thing
            interval.text = lyric_lines 
        tg.addTier(myTier,[2],"warning")
        tg.save(fn,'long_textgrid')

        #tg.addTier(input_tg,output_tg,"phrase")
#file format: NAME + "beat"
def beats_to_textgird(input_wav):

        name = os.path.splitext(fn)[0]
        #remove "beat" so it matches song file
        tgFn = name.strip("beat") + ".TextGrid"
        praat_scripts.annotateSilences(
         praat_exe, join(input_wavs, fn), join(input_tg, tgFn),100,0,-25,0.1,0.1,SILENCE_LABEL,SOUND_LABEL
        )

#def addTier(input_tg, output_tg,tier_label):
        # tg = textgrid.openTextgrid(join(input_tg, fn),True)
        # myTier = textgrid.IntervalTier(tier_label, entryList= [('0', tg.maxTimestamp,
        # 'test'),],minT=0,maxT=tg.maxTimestamp)a
        # tg.addTier(myTier)
        # tg.save(join(output_tg,fn),'long_textgrid', True)
#finds the txt file that matches the         
def add_lines_intervals(input_tg,input_txt,tier_num):
        tg = textgrid.openTextgrid(input_tg,True)
        myTier = tg.tierDict[tier_num]
        lyrics = open(input_txt,"r")
        lyricList = lyrics.readlines()
        dur = (tg.maxTimestamp/len(lyricList))
        start= 0.0
        length = start + dur
        line = 0
        while length <= tg.maxTimestamp:
            tmpInterval = Interval(start,length,lyricList[line])
            myTier.insertEntry(tmpInterval,'replace','warning')
            start = start+dur
            length = length +dur 
            line = line + 1
        tg.save(join(input, fn),'long_textgrid',True)



        
   




#autoSegmentSpeech(praatEXE, inputWavPath, input_tg)
#markIctus(praatEXE,inputWavPath,input_tg)
#addPhraseTier(input_tg, finalTGPath)
addIntervalsLyrics(finalTGPath)
#getLyrics(input_tg)